# spaCy と LLM で抽出したトリプルの比較をする

In [1]:
from pathlib import Path
import json

In [2]:
data_dir = Path("../data/phase_2_outputs")

sample_text_file = data_dir / "sample_texts.txt"
triples_llm_file = data_dir / "triples_llm.json"
triples_spacy_file = data_dir / "triples_spacy.json"

In [3]:
with open(sample_text_file, "r", encoding="utf-8") as f:
    texts = [
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith("#")
    ]

In [4]:
with open(triples_llm_file, "r", encoding="utf-8") as f:
    triples_llm = json.load(f)

with open(triples_spacy_file, "r", encoding="utf-8") as f:
    triples_spacy = json.load(f)

## 正規化関数

spaCyの結果は、snake case になってなかったりするので、正規化する関数を通して比較できるようにする。
この処理は実際に必要になるでしょう。

In [5]:
from typing import Dict, Tuple

TripleDict = Dict[str, str]
TripleTuple = Tuple[str, str, str]

def normalize_token(token: str) -> str:
    """小文字化+snake_case 化など。LLM側と揃える。"""
    token = token.strip().lower()
    return token.replace(" ", "_")

def normalize_triple(trp: TripleDict) -> TripleTuple:
    return (
        normalize_token(trp["subject"]),
        normalize_token(trp["verb"]),
        normalize_token(trp["object"]),
    )


## 取り込んだトリプルを正規化してsetにする

In [6]:
normalized_triples_llm = {normalize_triple(t) for t in triples_llm}
normalized_triples_spacy = {normalize_triple(t) for t in triples_spacy}

## 差分を確認

### 入力テキストを確認

In [7]:
for t in texts:
    print(t)

Logistic Regression is a model widely used for binary classification.
Random Forest is an ensemble model of many decision trees.
Transformer is a model based on the attention mechanism.
GPT is a Transformer-based model that uses only the decoder block.


### 共通のトリプル

In [8]:
normalized_triples_llm & normalized_triples_spacy

{('logistic_regression', 'be', 'model'),
 ('model', 'base_on', 'attention_mechanism'),
 ('random_forest', 'be', 'ensemble_model'),
 ('transformer', 'be', 'model')}

### spaCyにだけある

In [9]:
normalized_triples_spacy - normalized_triples_llm

{('gpt', 'be', 'base_model'), ('model', 'use_for', 'binary_classification')}

### LLMの方にだけある

In [10]:
normalized_triples_llm - normalized_triples_spacy

{('gpt', 'be', 'transformer_based_model'),
 ('gpt', 'use', 'decoder_block'),
 ('logistic_regression', 'use_for', 'binary_classification')}

## 結果

- 1文目の抽出結果に顕著な特徴が出ている
  - LLMでは `('logistic_regression', 'use_for', 'binary_classification')`
  - spaCyでは `('model', 'use_for', 'binary_classification')`
  - spaCyは文章の構造を忠実に再現している。しかし、LLMでの抽出は文章の意味を汲んだうえで抽出している
  - 文書構造を正確にグラフにするなら spaCy の結果が好ましいが、知識グラフを構築し、GraphRAGに適用するならLLMでの抽出が向いていそう
    - spaCyの結果でグラフを作ると、細かいノードの接続が大量に増えることが想像できる
    - 特定の知識にたどり着くためにたくさんのトリプルを使う必要が出てきそう（文章の流れとしては正確かもしれないが）
    - 一方LLMでは、意味を圧縮して知識を構造化する傾向にあるので、無駄が少なくなる（と思う）
